In [1]:
import geocoder
import functools as fn

In [2]:
from collections import Counter

In [31]:
try:
    from tqdm import tqdm
    STATUS_tqdm = True
except ImportError:
    STATUS_tqdm = False
    print('tqdm is not installed')
    print('Please install tqdm to check progress')
    print('run "!pip install tqdm" to install tqdm in jupyter')

In [8]:
status_tqdm = False

In [ ]:
?geocoder.google

In [ ]:
sample = geocoder.google('경기도 성남시 분당구 정자1동')

In [ ]:
sample.json

In [ ]:
sample2 = geocoder.google('경기도 성남시 분당구 정자1동 엠코헤리츠 101동 102호')

In [ ]:
sample2.json

In [ ]:
sample2.error

In [ ]:
sample.error is None

In [ ]:
sample2.error is not None

In [ ]:
2**12

In [ ]:
sample.latlng

In [3]:
class GeocodingError(Exception):
    '''
    Google Geocoding API의 에러를 받아서 raise시킨다.
    
    OVER_QUERY_LIMIT : 구글 API의 일일 제한 한도를 초과할 경우 발생하는 에러
    '''
    
    def __init__(self, error_type):
        self.error_type = error_type

In [ ]:
raise Exception('a', 'bb')

In [ ]:
?Exception

In [4]:
@fn.lru_cache(maxsize=2500)
def m_geocode(addr):
    # print('Live Geocoding!')
    r = geocoder.google(addr)
    
    if r.error is not None:
        raise GeocodingError(r.error)
    
    return r

In [ ]:
m_geocode('경기도 성남시 분당구 정자1동 엠코헤리츠 101동 102호')

In [ ]:
try:
    m_geocode('경기도 성남시 분당구 정자1동 엠코헤리츠 101동 102호')
except GeocodingError as e:
    aaa = e

In [ ]:
aaa.args[0]

첫번째에는 구글 api를 통해 데이터를 불러온다

In [ ]:
m_geocode('경기도 성남시 분당구 정자1동')

두 번째로 같은 위치를 요청하면 캐시된 데이터를 불러온다

In [ ]:
m_geocode('경기도 성남시 분당구 정자1동')

In [5]:
def geocode_list(addr_list):
    # 최종적으로 결과물을 저장할 리스트
    result_list = []
    
    # 에러가 발생할 경우 에러 종류를 저장할 리스트
    error_list = []
    
    if status_tqdm:
        loop = tqdm(addr_list)
    else:
        loop = addr_list
    
    for addr in loop:
        try:
            # geocoding 시도
            g = m_geocode(addr)
            result_list.append({'addr': addr, 'lat': g.latlng[0], 'lng': g.latlng[1]})
        except GeocodingError as e:
            # 에러가 발생할 경우 None으로 값을 채운다
            result_list.append({'addr': addr, 'lat': None, 'lng': None})
            error_list.append(e.args[0])

    # 에러가 발생했을 경우 에러별로 횟수를 정리해서 공유
    if len(error_list) > 0:
        error_counts = Counter(error_list)
        print('Error Counts : ', error_counts.most_common())
        
    return result_list

In [ ]:
c = Counter(['a', 'a', 'a', 'b', 'b'])

In [ ]:
print('Error Counts : ', c.most_common())

In [ ]:
[x for x in c.fromkeys()]

In [ ]:
for e in c.keys():
    print(e)

In [ ]:
len([])

In [6]:
sample_list = ['경기도 성남시 분당구 정자1동 엠코헤리츠 101동 102호', 
               '경기도 성남시 분당구 정자1동 엠코헤리츠 101동 102호',
               '경기도 성남시 분당구 정자1동 엠코헤리츠 101동 102호',
               '경기도 성남시 분당구 정자1동 엠코헤리츠 101동 102호',
               '경기도 성남시 분당구 정자1동',
                '서울시 서대문구 신촌동']

In [70]:
sample_list2 = ['경기도 성남시 분당구 정자1동',
                '서울시 서대문구 신촌동',
              '경기도 성남시 분당구 정자1동',
                '서울시 서대문구 신촌동']

In [9]:
geocode_list(sample_list)

Error Counts :  [('ZERO_RESULTS', 4)]


[{'addr': '경기도 성남시 분당구 정자1동 엠코헤리츠 101동 102호', 'lat': None, 'lng': None},
 {'addr': '경기도 성남시 분당구 정자1동 엠코헤리츠 101동 102호', 'lat': None, 'lng': None},
 {'addr': '경기도 성남시 분당구 정자1동 엠코헤리츠 101동 102호', 'lat': None, 'lng': None},
 {'addr': '경기도 성남시 분당구 정자1동 엠코헤리츠 101동 102호', 'lat': None, 'lng': None},
 {'addr': '경기도 성남시 분당구 정자1동', 'lat': 37.3614515, 'lng': 127.111435},
 {'addr': '서울시 서대문구 신촌동', 'lat': 37.5646027, 'lng': 126.9390819}]

In [30]:
True and True

True

In [29]:
True and False

False

In [72]:
class GeocodeByGoogle:
    def __init__(self, show_progress=True):
        self.status_tqdm = STATUS_tqdm and show_progress
    
    @fn.lru_cache(maxsize=2500)
    def geocode(self, addr):
        # print('live geocoding!')
        r = geocoder.google(addr)

        if r.error is not None:
            raise GeocodingError(r.error)

        return r
    
    def geocode_list(self, addr_list):
        # 최종적으로 결과물을 저장할 리스트
        result_list = []

        # 에러가 발생할 경우 에러 종류를 저장할 리스트
        error_list = []

        if self.status_tqdm:
            loop = tqdm(addr_list)
        else:
            loop = addr_list

        for addr in loop:
            try:
                # geocoding 시도
                g = self.geocode(addr)
                result_list.append({'addr': addr, 'lat': g.latlng[0], 'lng': g.latlng[1]})
            except GeocodingError as e:
                # 에러가 발생할 경우 None으로 값을 채운다
                result_list.append({'addr': addr, 'lat': None, 'lng': None})
                error_list.append(e.args[0])

        # 에러가 발생했을 경우 에러별로 횟수를 정리해서 공유
        if len(error_list) > 0:
            error_counts = Counter(error_list)
            print('Error Counts : ', error_counts.most_common())

        return result_list
    
    def cache_clear(self):
        self.geocode.cache_clear()
        print('Cache Cleared!')
        
    def cache_info(self):
        return self.geocode.cache_info()

In [51]:
aa = m_geocode.cache_info()

In [52]:
aa

CacheInfo(hits=0, misses=0, maxsize=2500, currsize=0)

In [73]:
gg = GeocodeByGoogle()

In [76]:
gg.geocode_list(sample_list)

100%|██████████| 6/6 [00:01<00:00,  5.32it/s]

Error Counts :  [('ZERO_RESULTS', 4)]


[{'addr': '경기도 성남시 분당구 정자1동 엠코헤리츠 101동 102호', 'lat': None, 'lng': None},
 {'addr': '경기도 성남시 분당구 정자1동 엠코헤리츠 101동 102호', 'lat': None, 'lng': None},
 {'addr': '경기도 성남시 분당구 정자1동 엠코헤리츠 101동 102호', 'lat': None, 'lng': None},
 {'addr': '경기도 성남시 분당구 정자1동 엠코헤리츠 101동 102호', 'lat': None, 'lng': None},
 {'addr': '경기도 성남시 분당구 정자1동', 'lat': 37.3614515, 'lng': 127.111435},
 {'addr': '서울시 서대문구 신촌동', 'lat': 37.5646027, 'lng': 126.9390819}]

In [77]:
gg.cache_info()

CacheInfo(hits=2, misses=2, maxsize=2500, currsize=2)

In [78]:
gg.geocode_list(sample_list2)

100%|██████████| 4/4 [00:00<00:00, 20092.47it/s]


[{'addr': '경기도 성남시 분당구 정자1동', 'lat': 37.3614515, 'lng': 127.111435},
 {'addr': '서울시 서대문구 신촌동', 'lat': 37.5646027, 'lng': 126.9390819},
 {'addr': '경기도 성남시 분당구 정자1동', 'lat': 37.3614515, 'lng': 127.111435},
 {'addr': '서울시 서대문구 신촌동', 'lat': 37.5646027, 'lng': 126.9390819}]

In [79]:
gg.cache_info()

CacheInfo(hits=6, misses=2, maxsize=2500, currsize=2)

In [80]:
gg.cache_clear()

Cache Cleared!


In [81]:
gg.cache_info()

CacheInfo(hits=0, misses=0, maxsize=2500, currsize=0)